# Comparing Electron Repulsion Integrals: pyquante2 vs MolecularIntegrals.jl

This notebook compares electron repulsion integrals (ERIs) for the H2O molecule computed using:
1. **pyquante2** - Python quantum chemistry package
2. **MolecularIntegrals.jl** - Julia package accessed via Python wrapper

Both calculations use the STO-3G basis set.

In [ ]:
import numpy as np
import time

## Part 1: ERIs using pyquante2

First, let's compute the electron repulsion integrals for H2O using pyquante2.

In [ ]:
from pyquante2.geo.molecule import molecule
from pyquante2.basisset import basisset
from pyquante2.ints.integrals import twoe_integrals

# Create H2O molecule
# Geometry in Angstroms (pyquante2 default)
# O at origin, H atoms at ~104.5 degree angle
h2o_pq = molecule([
    (8, 0.0, 0.0, 0.0),      # Oxygen
    (1, 0.757, 0.586, 0.0),  # Hydrogen 1
    (1, -0.757, 0.586, 0.0), # Hydrogen 2
], units='Angstrom')

print("H2O Molecule:")
print(h2o_pq)

In [ ]:
# Build STO-3G basis set
bfs_pq = basisset(h2o_pq, 'sto-3g')

print(f"Basis set: STO-3G")
print(f"Number of basis functions: {len(bfs_pq)}")

In [ ]:
# Compute all two-electron integrals
print("Computing ERIs with pyquante2...")
start = time.time()
eris_pq = twoe_integrals(bfs_pq)
pq_time = time.time() - start

print(f"Time: {pq_time:.4f} seconds")
print(f"ERI tensor shape: {eris_pq._2e_ints.shape}")

# Show some statistics
print(f"\nIntegral statistics:")
print(f"  Max:  {np.max(eris_pq._2e_ints):.10f}")
print(f"  Min:  {np.min(eris_pq._2e_ints):.10f}")
print(f"  Mean: {np.mean(eris_pq._2e_ints):.10f}")
print(f"  Std:  {np.std(eris_pq._2e_ints):.10f}")

In [ ]:
# Show some specific integrals
print("Some specific ERIs from pyquante2:")
print(f"  (0,0|0,0) = {eris_pq._2e_ints[0,0,0,0]:.10f}")
print(f"  (0,0|1,1) = {eris_pq._2e_ints[0,0,1,1]:.10f}")
print(f"  (0,1|0,1) = {eris_pq._2e_ints[0,1,0,1]:.10f}")
print(f"  (1,1|1,1) = {eris_pq._2e_ints[1,1,1,1]:.10f}")
print(f"  (2,2|2,2) = {eris_pq._2e_ints[2,2,2,2]:.10f}")

## Part 2: ERIs using MolecularIntegrals.jl

Now let's compute the same integrals using the Julia package via the Python wrapper.

In [ ]:
from molecular_integrals import Atom, Basis, MolecularIntegrals

# Create H2O molecule with same geometry
# Convert coordinates from Angstroms to Bohr (1 Angstrom = 1.8897259886 Bohr)
bohr_per_angstrom = 1.8897259886

atoms = [
    Atom(8, (0.0, 0.0, 0.0)),                                          # O at origin
    Atom(1, (0.757 * bohr_per_angstrom, 0.586 * bohr_per_angstrom, 0.0)),  # H
    Atom(1, (-0.757 * bohr_per_angstrom, 0.586 * bohr_per_angstrom, 0.0)), # H
]

print("H2O Molecule (coordinates in Bohr):")
for i, atom in enumerate(atoms):
    print(f"  Atom {i}: {atom}")

In [ ]:
# Build STO-3G basis set
basis = Basis(atoms, "sto3g")

print(f"Basis set: {basis}")
print(f"Number of basis functions: {len(basis)}")

In [ ]:
# Create integral engine
mi = MolecularIntegrals()

# Compute all two-electron integrals
print("Computing ERIs with MolecularIntegrals.jl...")
print("(Note: First call may be slow due to Julia JIT compilation)")
start = time.time()
all_eris_jl = mi.all_twoe_ints(basis)
jl_time = time.time() - start

print(f"Time: {jl_time:.4f} seconds")
print(f"Number of unique integrals (1D packed): {len(all_eris_jl)}")

# Get 4D tensor for easier comparison
eris_jl_4d = mi.all_twoe_ints_4d(basis)
print(f"ERI tensor shape: {eris_jl_4d.shape}")

# Show some statistics
print(f"\nIntegral statistics:")
print(f"  Max:  {np.max(eris_jl_4d):.10f}")
print(f"  Min:  {np.min(eris_jl_4d):.10f}")
print(f"  Mean: {np.mean(eris_jl_4d):.10f}")
print(f"  Std:  {np.std(eris_jl_4d):.10f}")

In [ ]:
# Show some specific integrals
print("Some specific ERIs from MolecularIntegrals.jl:")
print(f"  (0,0|0,0) = {eris_jl_4d[0,0,0,0]:.10f}")
print(f"  (0,0|1,1) = {eris_jl_4d[0,0,1,1]:.10f}")
print(f"  (0,1|0,1) = {eris_jl_4d[0,1,0,1]:.10f}")
print(f"  (1,1|1,1) = {eris_jl_4d[1,1,1,1]:.10f}")
print(f"  (2,2|2,2) = {eris_jl_4d[2,2,2,2]:.10f}")

## Part 3: Comparison

Let's compare the results from both methods.

In [ ]:
# Compute differences
diff = np.abs(eris_pq._2e_ints - eris_jl_4d)
max_diff = np.max(diff)
mean_diff = np.mean(diff)
rms_diff = np.sqrt(np.mean(diff**2))

print("="*60)
print("Comparison: pyquante2 vs MolecularIntegrals.jl")
print("="*60)
print(f"\nDifference statistics:")
print(f"  Maximum difference: {max_diff:.2e}")
print(f"  Mean difference:    {mean_diff:.2e}")
print(f"  RMS difference:     {rms_diff:.2e}")

print(f"\nTiming comparison:")
print(f"  pyquante2:           {pq_time:.4f} seconds")
print(f"  MolecularIntegrals.jl: {jl_time:.4f} seconds")
print(f"  Speedup:             {pq_time/jl_time:.2f}x")

# Determine if results match
if max_diff < 1e-10:
    print("\n✓ PASS: Integrals agree to machine precision!")
elif max_diff < 1e-6:
    print("\n✓ PASS: Integrals agree to 6 decimal places")
else:
    print("\n⚠ WARNING: Significant differences found")

In [ ]:
# Find the largest differences
print("\nIntegrals with largest absolute differences:")
flat_diff = diff.flatten()
flat_pq = eris_pq._2e_ints.flatten()
flat_jl = eris_jl_4d.flatten()

# Get indices of 5 largest differences
top_indices = np.argsort(flat_diff)[-5:][::-1]

for idx in top_indices:
    # Convert flat index to 4D indices
    n = eris_pq._2e_ints.shape[0]
    i = idx // (n**3)
    j = (idx % (n**3)) // (n**2)
    k = (idx % (n**2)) // n
    l = idx % n
    
    print(f"  ({i},{j}|{k},{l}): pq={flat_pq[idx]:.10f}, jl={flat_jl[idx]:.10f}, diff={flat_diff[idx]:.2e}")

## Summary

This notebook demonstrated:
1. Computing ERIs for H2O using pyquante2 (pure Python)
2. Computing the same ERIs using MolecularIntegrals.jl (Julia via Python wrapper)
3. Comparing the results to verify accuracy

The results should match to machine precision, validating that the Julia implementation produces correct results while potentially offering better performance.